# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

[/home/runner/work/polars/polars/polars/polars-io/src/ndjson/core.rs:162] &data_type = Struct(
    [
        Field {
            name: "Context_Name",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_MaritalStatus",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_CLV",
            data_type: Int64,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Customer_City",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "IH_Web_Inbound_Accepted_pxLastGroupID",
            data_type: LargeUtf8,
            is_nullable: true,
            metadata: {},
        },
        Field {
            name: "Decision_Outcome",
            data_type: LargeUtf8,
            is_null

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""15243619332462…","""../../../../da…","""10149035484578…",1.2927e19,"""81576304423393…","""91376559699788…",false
"""10560957371051…","""../../../../da…","""11259203928390…",1.4856e19,"""81576304423393…","""10910283550035…",true
"""51409184502006…","""../../../../da…","""18388052367913…",5.6458e17,"""17146062868891…","""91376559699788…",false
"""10560957371051…","""../../../../da…","""12324963588222…",4.0723e18,"""66398332728223…","""91376559699788…",false
"""15243619332462…","""../../../../da…","""10296852847910…",1.4677e19,"""66398332728223…","""13271899037682…",true
"""51409184502006…","""../../../../da…","""13304500656889…",1.6330e19,"""12365283327124…",null,false
"""51409184502006…","""../../../../da…","""43179294262795…",8.7675e18,"""66398332728223…","""13271899037682…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""57981767698644…","""../../../../da…","""39014622327347…",1.2927e19,"""FirstMortgage3…","""29864510279585…","""Rejected"""
"""20034400615145…","""../../../../da…","""13506148910785…",1.4856e19,"""FirstMortgage3…","""29321842687390…","""Accepted"""
"""17003349692390…","""../../../../da…","""17868425778774…",5.6458e17,"""MoneyMarketSav…","""29864510279585…","""Rejected"""
"""20034400615145…","""../../../../da…","""38088291581577…",4.0723e18,"""BasicChecking""","""29864510279585…","""Rejected"""
"""57981767698644…","""../../../../da…","""14077697359937…",1.4677e19,"""BasicChecking""","""59590284364673…","""Accepted"""
"""17003349692390…","""../../../../da…","""34628527521707…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""17003349692390…","""../../../../da…","""15609339381971…",8.7675e18,"""BasicChecking""","""59590284364673…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""12154556426016…","""../../../../da…","""51519854107079…",1.2927e19,"""FirstMortgage3…","""15929775116617…","""Rejected"""
"""87092951009805…","""../../../../da…","""15153075939479…",1.4856e19,"""FirstMortgage3…","""22217454315621…","""Accepted"""
"""37034796642100…","""../../../../da…","""53797203847259…",5.6458e17,"""MoneyMarketSav…","""15929775116617…","""Rejected"""
"""87092951009805…","""../../../../da…","""13115112908575…",4.0723e18,"""BasicChecking""","""15929775116617…","""Rejected"""
"""12154556426016…","""../../../../da…","""45128932187154…",1.4677e19,"""BasicChecking""","""14102865238668…","""Accepted"""
"""37034796642100…","""../../../../da…","""13714943107391…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""37034796642100…","""../../../../da…","""12843442113734…",8.7675e18,"""BasicChecking""","""14102865238668…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
filename=filename
Customer_City=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""18316296074826…","""13446682987184…",1.2927e19,"""FirstMortgage3…","""29800245460062…","""Rejected"""
"""10164741166500…","""13141756321301…",1.4856e19,"""FirstMortgage3…","""49548930210707…","""Accepted"""
"""16140911506523…","""16428835737377…",5.6458e17,"""MoneyMarketSav…","""29800245460062…","""Rejected"""
"""10164741166500…","""15030230866027…",4.0723e18,"""BasicChecking""","""29800245460062…","""Rejected"""
"""18316296074826…","""16001436589638…",1.4677e19,"""BasicChecking""","""76436698215230…","""Accepted"""
"""16140911506523…","""92850967457124…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""16140911506523…","""92249694172767…",8.7675e18,"""BasicChecking""","""76436698215230…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'